In [1]:
import numpy as np

def complex_simulator(x):
    """
    Simulates a more complex product design.

    Args:
    - x (array-like): A 1D array representing the design parameters.

    Returns:
    - feasible (int): 0 if the design is infeasible, 1 otherwise.
    - y (array-like): A 1D array representing the product properties. Meaningful only if feasible=1.
    """

    assert len(x) == 10, "Design parameter vector must have 10 elements."

    # Complex non-convex feasibility condition
    if (np.sum(x[:5]**2) - 10) < 0 or np.prod(x[5:]) < 0:
        return 0, None

    # If feasible, compute product properties using some non-linear function
    y1 = np.sin(np.prod(x[:3])) + x[3]**2 - x[4]
    y2 = x[5]**3 - np.cos(x[6]*x[7])
    y3 = x[8]**2 * np.tan(x[9])
    y4 = np.dot(x[:5], x[5:])
    y5 = np.prod(x[:3]) - np.sum(x[7:])
    y = np.array([y1, y2, y3, y4, y5])

    return 1, y

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# Generate data
N = 10000
x_data = np.random.rand(N, 10) * 2 - 1  # Random data in range [-1, 1]
y_data = []
feasibility = []

for x in x_data:
    f, y = complex_simulator(x)
    feasibility.append(f)
    if f == 1:
        y_data.append(y)
    else:
        y_data.append(np.zeros(5))  # Filling in with zeros for infeasible solutions

y_data = np.array(y_data)

# Convert data to torch tensors
x_tensor = torch.tensor(x_data, dtype=torch.float32)
y_tensor = torch.tensor(y_data, dtype=torch.float32)

def set_device(use_gpu):
    if not use_gpu:
        device = torch.device('cpu')
    elif torch.cuda.is_available():
        device = torch.device('cuda')
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    elif not torch.backends.mps.is_available():
        if not torch.backends.mps.is_built():
            raise EnvironmentError("MPS not available because the current PyTorch install was not "
                                   "built with MPS enabled.")
        else:
            device = torch.device('cpu')
            Warning("Cannot use GPU. Please check for CUDA/Mac OS version. Using CPU instead.")
    return device

In [4]:
from sklearn.metrics import r2_score

# Define a multi-task ANN
class FeasibilityNet(nn.Module):
    def __init__(self):
        super(FeasibilityNet, self).__init__()
        self.fc1 = nn.Linear(10, 20)
        self.fc2 = nn.Linear(20, 20)
        # Regression head
        self.regression_head = nn.Linear(20, 5)
        # Classification head
        self.classification_head = nn.Linear(20, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        reg_out = self.regression_head(x)
        class_out = self.sigmoid(self.classification_head(x))
        return class_out, reg_out

model = FeasibilityNet()
regression_criterion = nn.MSELoss()
classification_criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

feasibility_tensor = torch.tensor(feasibility, dtype=torch.float32).view(-1, 1)

device = set_device(True)
print(f"The device is set to {device}")

model = model.to(device)
x_tensor = x_tensor.to(device)
y_tensor = y_tensor.to(device)
feasibility_tensor = feasibility_tensor.to(device)

# Train the model
epochs = 1200
for epoch in range(epochs):
    class_out, reg_out = model(x_tensor)
    reg_loss = regression_criterion(reg_out, y_tensor)
    class_loss = classification_criterion(class_out, feasibility_tensor)
    # Combine the losses. Weights can be adjusted as needed.
    loss = reg_loss + class_loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 400 == 0:
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")

# Accuracy
with torch.no_grad():
    class_predictions, reg_predictions = model(x_tensor)
    predicted_labels = (class_predictions > 0.5).float()
    correct = (predicted_labels == feasibility_tensor).float().sum()
    accuracy = correct / feasibility_tensor.size(0)

    mse_loss = regression_criterion(reg_predictions, y_tensor).item()
    r2 = r2_score(y_tensor.cpu().numpy(), reg_predictions.cpu().numpy())



print(f"Classification Accuracy: {accuracy:.4f}")
print(f"Final Mean Squared Error: {mse_loss:.4f}")
print(f"R^2 Score: {r2:.4f}")


y_true = y_tensor.cpu().numpy()
y_pred = reg_predictions.cpu().numpy()

ss_res = np.sum((y_true - y_pred) ** 2)
ss_tot = np.sum((y_true - y_true.mean()) ** 2)
r2_manual = 1 - (ss_res / ss_tot)

print(f"Manual R^2 Score: {r2_manual:.4f}")


The device is set to cuda
Epoch [0/1200], Loss: 0.7687
Epoch [400/1200], Loss: 0.0020
Epoch [800/1200], Loss: 0.0005
Classification Accuracy: 1.0000
Final Mean Squared Error: 0.0001
R^2 Score: 0.0000
Manual R^2 Score: -inf


C:\Users\justi\AppData\Local\Temp\ipykernel_14968\1244601412.py:73: RuntimeWarning: divide by zero encountered in scalar divide
  r2_manual = 1 - (ss_res / ss_tot)
